In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Get and transform test data (Long Beach Transit, 170)

In [2]:
lbt_itp_id = 170

In [3]:
# %%sql -o lbt_positions

# # get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
# SELECT *
# FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
# WHERE _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_170_0.parquet"
#     OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-11-30_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-03_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-04_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-05_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-06_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-07_170_0.parquet"
# ORDER BY header_timestamp

#### Reformat timestamps

In [4]:
# ## set system time
# os.environ['TZ'] = 'America/Los_Angeles'
# time.tzset()
# time.tzname

In [5]:
# lbt_positions.vehicle_timestamp = lbt_positions.vehicle_timestamp.apply(convert_ts)
# lbt_positions.header_timestamp = lbt_positions.header_timestamp.apply(convert_ts)

In [6]:
# lbt_positions.to_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [7]:
lbt_positions = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [8]:
pch_routes = ('171', '172', '173', '174', '175')

In [9]:
# min_date, max_date = ('2021-11-01', '2021-11-30')

# lbt_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
#     # >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
#     >> filter(_.calitp_itp_id == lbt_itp_id)
#     >> filter(_.service_date == max_date)
#     >> filter(_.is_in_service == True)
#     >> select(_.trip_key, _.service_date)
#     >> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
#     >> select(_.calitp_itp_id, _.calitp_url_number, _.service_date,
#               _.trip_key, _.trip_id, _.route_id, _.direction_id,
#               _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
#     # >> filter(_.route_id.isin(pch_routes))

#     >> collect()
#     )

In [10]:
# lbt_trips.to_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [11]:
lbt_trips = pd.read_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [12]:
# stop_times = (tbl.gtfs_schedule.stop_times()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      >> arrange(_.stop_sequence)
#      >> collect()
#     )

In [13]:
# stop_times.to_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [14]:
stop_times = pd.read_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [15]:
# stops = (tbl.gtfs_schedule.stops()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      # >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      # >> arrange(_.stop_sequence)
#      >> select(_.calitp_itp_id, _.calitp_url_number, _.stop_id,
#               _.stop_lat, _.stop_lon, _.stop_name)
#      >> collect()
#     )

# stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
#                         crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [16]:
# shared_utils.utils.geoparquet_gcs_export(stops, GCS_FILE_PATH, 'lbt_stops')

In [17]:
stops = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_stops.parquet')

In [18]:
trips_positions_joined = (lbt_trips
                        # >> filter(_.route_id.isin(pch_routes))
                        >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id', 'calitp_url_number'])
                       )

In [19]:
trips_positions_joined.vehicle_timestamp.max()

Timestamp('2021-12-01 11:15:59')

In [20]:
trips_positions_joined.vehicle_timestamp.min()

Timestamp('2021-11-29 15:57:13')

In [21]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [22]:
# lbt_routelines = shared_utils.geography_utils.make_routes_shapefile(['170'], ## LBT
#                                 shared_utils.geography_utils.CA_NAD83Albers)

In [23]:
# shared_utils.utils.geoparquet_gcs_export(lbt_routelines, GCS_FILE_PATH, 'lbt_routelines')

In [24]:
lbt_routelines = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_routelines.parquet')

In [25]:
trips_positions_joined = trips_positions_joined >> filter(_.vehicle_timestamp.apply(lambda x: x.day == 30))

In [26]:
trips_positions_joined.head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
285,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:50:38,POINT (174524.511 -468602.082)
286,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:50:38,POINT (174524.511 -468602.082)
287,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:51:08,POINT (174524.511 -468602.082)


In [27]:
stop_times.head(3)

,calitp_itp_id,calitp_url_number,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at
0,170,0,9285982,0666,1,10:41:00,10:41:00,None,0,0,None,None,0.0,1,2021-12-17
1,170,0,9288986,4003,1,17:05:00,17:05:00,None,0,1,None,None,0.0,1,2021-12-17
2,170,0,9286129,4012,1,14:20:00,14:20:00,None,0,1,None,None,0.0,1,2021-12-17


## Vehicle Positions Trip class

#### Examples

In [28]:
example_trip = (trips_positions_joined
 >> filter(_.trip_key == 1553149201202261125)
)

In [29]:
example_trip.head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
327364,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:02:37,POINT (174550.467 -468602.675)
327365,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:02:37,POINT (174550.467 -468602.675)
327366,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:03:07,POINT (174550.467 -468602.675)


In [30]:
example_vp_trip = rt.VehiclePositionsTrip(example_trip, lbt_routelines)

sc_called
(59, 4)


In [31]:
example_vp_trip.vehicle_positions.head(3)

,vehicle_timestamp,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
0,2021-11-30 17:02:12,2021-11-30 17:02:37,POINT (174550.467 -468602.675),74.936069,NaT,NaN,NaN,NaN,False,NaN
1,2021-11-30 17:03:13,2021-11-30 17:03:37,POINT (174199.373 -468606.652),426.501155,2021-11-30 17:02:12,74.936069,61.0,351.565087,True,5.763362
2,2021-11-30 17:04:13,2021-11-30 17:04:37,POINT (173950.148 -468607.933),676.459161,2021-11-30 17:03:13,426.501155,60.0,249.958006,True,4.165967


In [32]:
example_vp_trip.time_at_position(550) ## interpolate time at 550 meters from start of route (shape_meters in df^)

Timestamp('2021-11-30 17:03:42.644702')

In [34]:
# example_vp_trip.detailed_speed_map()

## General RT Analysis class

In [35]:
example_trips = (trips_positions_joined
 >> filter(_.route_id == '175', _.direction_id == '0') ## towards Villages at Cabrillo
 >> group_by(_.trip_key)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 >> head(6) ## afternoon/evening
)
example_trips

,trip_key,max_time
4,-5374575435373645071,2021-11-30 07:14:16
12,3082123411846948636,2021-11-30 07:54:17
0,-8296488155279914403,2021-11-30 08:38:01
1,-7508921904019176003,2021-11-30 09:15:30
14,4908653065062143760,2021-11-30 09:54:22
5,-4201736542695259394,2021-11-30 10:35:43


In [36]:
%%capture
## suppress warnings from location data inaccuracies
## generate test analysis with a small subset of LBT trips on a particular route, time, direction
test_analysis = rt.RtAnalysis(trips_positions_joined, stop_times, stops, lbt_routelines, example_trips.trip_key)

In [37]:
%%capture
## suppress warnings from start/end of routes
all_delays = test_analysis.generate_delay_view()

KeyError: 'stop_sequence'

In [35]:
all_delays.head(3)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry
145,2021-11-30 07:02:41,2021-11-30 07:03:22.362317,0 days 00:00:41.362317,1027,9288593,1750012,0,Westbound,2,175,501.040761,2021-12-01,POINT (174124.462 -468604.478)
144,2021-11-30 08:21:41,2021-11-30 08:25:52.371129,0 days 00:04:11.371129,1027,9288597,1750012,0,Westbound,2,175,501.040761,2021-12-01,POINT (174124.462 -468604.478)
148,2021-11-30 09:44:55,2021-11-30 09:44:21.605429,-1 days +23:59:26.605429,1027,9288601,1750012,0,Westbound,2,175,501.040761,2021-12-01,POINT (174124.462 -468604.478)


In [36]:
all_summary = test_analysis.generate_delay_summary()

In [42]:
all_summary.head(5)

,stop_id,geometry,stop_sequence,avg_delay,max_delay
0,1027,POINT (174124.462 -468604.478),2,0 days 00:01:26.446291666,0 days 00:04:11.371129
1,0540,POINT (173921.407 -468897.107),3,0 days 00:01:07.972914400,0 days 00:04:28.865438
2,0542,POINT (173328.363 -468647.513),4,0 days 00:00:39.873642666,0 days 00:03:58.590881
3,0543,POINT (173108.288 -468477.715),5,0 days 00:00:37.971737666,0 days 00:03:32.980943
4,0544,POINT (173001.551 -468534.279),6,0 days 00:01:56.187369166,0 days 00:03:37.006580


In [37]:
# test_analysis.map_stop_delays(how = 'average') ## to refine, add hover, legend, etc...

In [38]:
# test_analysis.map_segment_speeds(how = 'low_speeds')

In [39]:
# test_analysis.map_stop_delays()

## Sandbox

In [40]:
trips_positions_joined >> filter(_.direction_id == '0') >> count(_.trip_id)

,trip_id,n
0,9288152,201
1,9288153,326
2,9288154,283
3,9288155,276
4,9288156,276
...,...,...
834,9296933,63
835,9296934,106
836,9296935,69
837,9297176,68


In [41]:
trips_positions_joined.vehicle_timestamp.iloc[0].day

30

In [46]:
pm_peak = (trips_positions_joined
 >> filter(_.direction_id == '0') ## towards 
 >> filter((_.vehicle_timestamp.apply(lambda x: x.hour < 20))
           & (_.vehicle_timestamp.apply(lambda x: x.hour > 16))) ## 4-8pm
 >> mutate(timestamp_day = _.vehicle_timestamp.apply(lambda x: x.day))
 # >> mutate(timestamp_hour = _.vehicle_timestamp.apply(lambda x: x.hour))
 # >> filter((_.timestamp_hour < 20)
 #           & (_.timestamp_hour > 16)) ## 4-8pm >> group_by(_.trip_id)
 >> filter(_.timestamp_day == 30)
 >> group_by(_.trip_id)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 # >> head(40) ## afternoon/evening
)
pm_peak

,trip_id,max_time
0,9288170,2021-11-30 17:00:34
130,9289459,2021-11-30 17:01:25
121,9289404,2021-11-30 17:01:52
58,9288776,2021-11-30 17:02:20
190,9296915,2021-11-30 17:03:04
...,...,...
77,9288961,2021-11-30 19:59:52
166,9296637,2021-11-30 19:59:52
145,9289764,2021-11-30 19:59:54
7,9288179,2021-11-30 19:59:55


In [ ]:
test_many = rt.RtAnalysis(trips_positions_joined,
                           stop_times,
                           stops,
                           lbt_routelines,
                           pm_peak.trip_id)

sc_called
(98, 4)
sc_called
(80, 4)
sc_called
(69, 4)
sc_called
(41, 4)
check location data for trip 9288776
sc_called
(34, 10)
check location data for trip 9288776
sc_called
(31, 10)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(21, 4)
sc_called
(100, 4)
sc_called
(86, 4)
sc_called
(77, 4)
sc_called
(22, 4)
sc_called
(28, 4)
sc_called
(76, 4)
sc_called
(35, 4)
sc_called
(109, 4)
sc_called
(39, 4)
sc_called
(67, 4)
sc_called
(94, 4)
sc_called
(65, 4)
sc_called
(46, 4)
sc_called
(60, 4)
check location data for trip 9288621
sc_called
(41, 10)
check location data for trip 9288621
sc_called
(35, 10)
check location data for trip 9288621
sc_called
(32, 10)
check location data for trip 9288621
sc_called
(30, 10)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(103, 4)
sc_called
(71, 4)
sc_called
(58, 4)
sc_called
(89, 4)
sc_called
(75, 4)
check location data for trip 9296673
sc_called
(50, 10)
check location data for trip 9296673
sc_called
(45, 10)
check location data for trip 9296673
sc_called
(42, 10)
sc_called
(22, 4)
sc_called
(123, 4)
check location data for trip 9288525
sc_called
(89, 10)
check location data for trip 9288525
sc_called
(83, 10)


...

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(72, 4)
check location data for trip 9288631
sc_called
(56, 10)
check location data for trip 9288631
sc_called
(53, 10)
sc_called
(29, 4)
sc_called
(103, 4)
sc_called
(69, 4)
check location data for trip 9288630
sc_called
(53, 10)
check location data for trip 9288630
sc_called
(50, 10)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(49, 4)
sc_called
(63, 4)
sc_called
(42, 4)
sc_called
(65, 4)
sc_called
(89, 4)
check location data for trip 9288179
sc_called
(72, 10)
check location data for trip 9288179
sc_called
(69, 10)
sc_called
(80, 4)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [48]:
%%capture --no-stderr
test_many.generate_delay_view()

In [49]:
test_many.trip_ids.nunique()

195

In [50]:
test_many.delay_view.trip_id.nunique()

193

In [51]:
test_many.delay_view.shape_id.nunique()

39

In [52]:
test_many.delay_view.head(3)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry
5096,2021-11-30 15:23:15,2021-11-30 15:25:14.254940,0 days 00:01:59.254940,1573,9288170,1910295,0,Westbound,2,191,559.250839,2021-12-01,POINT (179249.800 -461991.129)
2651,2021-11-30 15:58:39,2021-11-30 15:56:35.872418,-1 days +23:57:56.872418,4146,9288525,920111,0,Southbound,2,92,919.583082,2021-12-01,POINT (174146.349 -456781.921)
2654,2021-11-30 16:38:39,2021-11-30 16:39:51.310372,0 days 00:01:12.310372,4146,9288528,920111,0,Southbound,2,92,919.583082,2021-12-01,POINT (174146.349 -456781.921)


In [53]:
df = (test_many.route_coverage_summary()
 >> mutate(stops_covered = _.max_stop - _.min_stop)
 >> arrange(-_.stops_covered)
 >> filter(_.stops_covered > 1)
 >> distinct(_.shape_id, _keep_all=True)
)
df.head(5)

,trip_id,shape_id,direction_id,min_meters,min_stop,max_meters,max_stop,stops_covered
0,9288535,910132,0,832.866019,2,25107.200799,80,78
1,9288529,910131,0,565.808266,3,24385.983516,80,77
2,9288641,1730123,0,785.141960,2,31511.120110,79,77
3,9288170,1910295,0,559.250839,2,27466.876359,76,74
4,9288217,1920167,0,1029.767210,3,26019.972326,77,74


In [50]:
# test_many.map_stop_delays() ## broken/inf recursion (delay view check failing?)

In [51]:
# test_many.map_segment_speeds(how = 'low_speeds')

In [56]:
# test_many.segment_speed_view.shape_id.nunique()

In [54]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [55]:
# m = test_many._generate_segment_map(how = 'low_speeds', colorscale = colorscale)

In [56]:
# m

In [57]:
# m = example_vp_trip.detailed_speed_map()

In [60]:
test_delay = test_many.delay_view.copy()

In [61]:
#TODO rewrite to not modify attributes!!

endpoint_delay_sum = (test_delay
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.max())
 >> ungroup()
 >> mutate(total_delay = _.delay.sum())
)

In [62]:
starts = (test_delay
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.min())
 >> ungroup()
 # >> mutate(total_delay = _.delay.sum())
 >> select(_.start_time == _.actual_time, _.trip_id, _.start_meters == _.shape_meters)
)

In [63]:
speed_and_delay = endpoint_delay_sum >> inner_join(_, starts, on='trip_id')

speed_and_delay['seconds'] = speed_and_delay.apply(
    lambda x: (x.actual_time - x.start_time).seconds, axis=1)

speed_and_delay['trip_speed'] = (speed_and_delay.shape_meters - speed_and_delay.start_meters) / speed_and_delay.seconds

speed_and_delay['trip_speed_mph'] = speed_and_delay.trip_speed * MPH_PER_MPS

speed_and_delay['trip_speed_mph'].mean()

12.388162329259766

In [64]:
speed_and_delay.head(3)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry,total_delay,start_time,start_meters,seconds,trip_speed,trip_speed_mph
0,2021-11-30 16:54:46,2021-11-30 17:01:10.276088,0 days 00:06:24.276088,4018,9296629,460067,0,Westbound,29,46,6580.90515,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 16:36:57.753146,798.756471,1452,3.982196,8.908173
1,2021-11-30 18:14:46,2021-11-30 18:12:56.658216,-1 days +23:58:10.658216,4018,9296633,460067,0,Westbound,29,46,6580.90515,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 17:52:27.617212,798.756471,1229,4.704759,10.524546
2,2021-11-30 19:13:46,2021-11-30 19:15:08.641013,0 days 00:01:22.641013,4018,9296636,460067,0,Westbound,29,46,6580.90515,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 18:52:55.190148,798.756471,1333,4.337696,9.703426


In [65]:
speed_and_delay >> mutate(total_mean_mph = (
    (_.shape_meters - _.start_meters).sum()
    / _.seconds.sum()) * MPH_PER_MPS)

,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,date,geometry,total_delay,start_time,start_meters,seconds,trip_speed,trip_speed_mph,total_mean_mph
0,2021-11-30 16:54:46,2021-11-30 17:01:10.276088,0 days 00:06:24.276088,4018,9296629,460067,0,Westbound,29,46,6580.905150,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 16:36:57.753146,798.756471,1452,3.982196,8.908173,11.882512
1,2021-11-30 18:14:46,2021-11-30 18:12:56.658216,-1 days +23:58:10.658216,4018,9296633,460067,0,Westbound,29,46,6580.905150,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 17:52:27.617212,798.756471,1229,4.704759,10.524546,11.882512
2,2021-11-30 19:13:46,2021-11-30 19:15:08.641013,0 days 00:01:22.641013,4018,9296636,460067,0,Westbound,29,46,6580.905150,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 18:52:55.190148,798.756471,1333,4.337696,9.703426,11.882512
3,2021-11-30 19:33:46,2021-11-30 19:33:39.347211,-1 days +23:59:53.347211,4018,9296637,460069,0,Westbound,29,46,6580.905150,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 19:14:54.846992,798.756471,1124,5.144260,11.507710,11.882512
4,2021-11-30 18:53:46,2021-11-30 18:54:05.667694,0 days 00:00:19.667694,4018,9296635,460067,0,Westbound,29,46,6580.905150,2021-12-01,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 18:35:30.624230,798.756471,1115,5.185784,11.600598,11.882512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2021-11-30 17:44:44,2021-11-30 17:58:43.638888,0 days 00:13:59.638888,2412,9289756,1210143,0,Westbound,49,121,16653.301676,2021-12-01,POINT (166657.821 -470795.826),0 days 08:49:11.823428,2021-11-30 16:53:37.061994,540.516202,3906,4.125137,9.227932,11.882512
189,2021-11-30 16:54:15,2021-11-30 16:49:50.777843,-1 days +23:55:35.777843,0081,9296673,450110,0,Westbound,36,45,10740.313325,2021-12-01,POINT (165309.129 -468459.853),0 days 08:49:11.823428,2021-11-30 16:17:14.681029,1599.712698,1956,4.673109,10.453744,11.882512
190,2021-11-30 17:33:15,2021-11-30 17:29:27.054533,-1 days +23:56:12.054533,0081,9296675,450110,0,Westbound,36,45,10740.313325,2021-12-01,POINT (165309.129 -468459.853),0 days 08:49:11.823428,2021-11-30 16:57:06.572425,1019.609190,1940,5.010672,11.208874,11.882512
191,2021-11-30 19:30:15,2021-11-30 19:26:15.931293,-1 days +23:56:00.931293,0081,9296681,450110,0,Westbound,36,45,10740.313325,2021-12-01,POINT (165309.129 -468459.853),0 days 08:49:11.823428,2021-11-30 18:55:51.028297,1019.609190,1824,5.329333,11.921719,11.882512
